In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

In [2]:
data = pd.read_csv('data.csv')
# texts = data['Sentence'].values
# data_temp = data.set_index('Sentiment')
# data = data_temp.drop('neutral').reset_index()
# data.head()

In [3]:
data['Sentence'].apply(len).max(), data['Sentence'].apply(len).min(), data['Sentence'].apply(len).median()

(315, 9, 107.0)

In [4]:
max_words = 10000
max_sequence_length = 315

In [5]:
text = data['Sentence']

In [6]:
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(text)
sequences = tokenizer.texts_to_sequences(text)

In [7]:
x = pad_sequences(sequences, maxlen=max_sequence_length, padding='post', truncating='post')
y = np.where(data['Sentiment']=='positive', 1, 0)
y.shape, x.shape

((5842,), (5842, 315))

In [8]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [9]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense

In [10]:
model = Sequential()

embedding_dim = 100  # Dimension of the word embeddings

model.add(Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=max_sequence_length))
model.add(LSTM(100))  # You can adjust the number of LSTM units
# model.add(LSTM(32))
model.add(Dense(1, activation='sigmoid'))  # Binary classification

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [11]:
epochs = 5  # Number of training epochs
batch_size = 100 # Batch size

model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)

Epoch 1/5
43/43 [==============================] - 48s 997ms/step - loss: 0.6333 - accuracy: 0.6866 - val_loss: 0.6468 - val_accuracy: 0.6538
Epoch 2/5
43/43 [==============================] - 44s 1s/step - loss: 0.6271 - accuracy: 0.6866 - val_loss: 0.6476 - val_accuracy: 0.6538
Epoch 3/5
43/43 [==============================] - 46s 1s/step - loss: 0.6223 - accuracy: 0.6866 - val_loss: 0.6466 - val_accuracy: 0.6538
Epoch 4/5
43/43 [==============================] - 45s 1s/step - loss: 0.6220 - accuracy: 0.6866 - val_loss: 0.6451 - val_accuracy: 0.6538
Epoch 5/5
43/43 [==============================] - 48s 1s/step - loss: 0.6229 - accuracy: 0.6866 - val_loss: 0.6459 - val_accuracy: 0.6538


In [12]:
adani = pd.read_excel('Sentiments_Adani.xlsx', parse_dates=True, sheet_name='Sentiments_Adani')
s = adani['Sentences']
seq_eval = tokenizer.texts_to_sequences(s)
x_eval = pad_sequences(seq_eval, padding='post', truncating='post')
y_eval = np.where(adani['Sentiments'] == 'Positive', 1, 0)

In [13]:
loss2, accuracy2 = model.evaluate(x_eval, y_eval)

1/1 [==============================] - 1s 1s/step - loss: 0.6985 - accuracy: 0.5714


In [14]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss:.4f}, Test Accuracy: {accuracy:.4f}")

37/37 [==============================] - 3s 92ms/step - loss: 0.6257 - accuracy: 0.6818
Test Loss: 0.6257, Test Accuracy: 0.6818


In [15]:
adani = pd.read_csv('Sentiments_Adani.csv', parse_dates=True)
adani.head()

,Date,Sentences,Sentiments,Returns(avg)
0,03-11-2023 12:20,Adani Green's profit increased to Rs 371 crore.,Positive,-0.31
1,02-11-2023 10:47,𝐅𝐈𝐈𝐬 𝐈𝐧𝐜𝐫𝐞𝐚𝐬𝐞𝐝 𝐒𝐭𝐚𝐤𝐞 𝐝𝐫𝐚𝐬𝐭𝐢𝐜𝐚𝐥𝐥𝐲 𝐢𝐧 𝐭𝐡𝐞𝐬𝐞 𝐬𝐭𝐨𝐜...,Positive,2.17
2,02-11-2023 08:24,Replugging: Adani group has clear links to How...,Negative,0.24
3,02-11-2023 05:26,Adani's wealth before Modi became CM = 3300 Cr...,Negative,-0.09
4,30-10-2023 15:01,A spate of recent media revelations has furthe...,Negative,-0.21


In [16]:
s = adani['Sentences']
sent = np.where(adani['Sentiments']=='Positive', 1, 0)

In [19]:
tokenizer.fit_on_texts(s)
seq_eval = tokenizer.texts_to_sequences(text)

In [20]:
x_eval = pad_sequences(seq_eval, maxlen=max_sequence_length, padding='post', truncating='post')

In [ ]:
y_eval = np.where(adani['Sentiments']=='Positive', 1, 0)

In [22]:
pred = model.predict(x_eval)

183/183 [==============================] - 17s 91ms/step


In [24]:
pred

5842

In [25]:
x_eval

array([[   1, 3980,  135, ...,    0,    0,    0],
       [5836,   12, 1311, ...,    0,    0,    0],
       [   7,    1,  129, ...,    0,    0,    0],
       ...,
       [ 110,  634,   14, ...,    0,    0,    0],
       [   3,    1,  165, ...,    0,    0,    0],
       [  78,  644, 3122, ...,    0,    0,    0]])